# Segmentation et Classification des Noyaux Cellulaires dans le Mélanome
## Projet Deep Learning

Dans ce projet, nous étudions la segmentation et classification des noyaux cellulaires dans les images histopathologiques du mélanome en utilisant des techniques de deep learning.

Nous nous basons sur l'article scientifique : "PUMA: A dataset for deep learning-based nuclei and tissue segmentation in advanced melanoma with improved biomarker potential" publié dans GigaScience en 2024.

Ce travail s'inscrit dans une démarche d'amélioration des biomarqueurs pronostiques pour le mélanome avancé.

# Contexte : Le Mélanome et ses Biomarqueurs

- Mélanome : cancer agressif de la peau en augmentation
- 50% des patients ne répondent pas à l'immunothérapie
- Les lymphocytes infiltrant la tumeur (TILs) = biomarqueur pronostique clé
- Limites actuelles : évaluation manuelle subjective et variable

Le mélanome est une forme particulièrement agressive de cancer cutané dont l'incidence augmente globalement. Bien que le mélanome primaire soit souvent traité par excision chirurgicale, le mélanome avancé nécessite une immunothérapie par inhibiteurs de points de contrôle immunitaires.

Malheureusement, environ la moitié des patients ne répondent pas à cette thérapie, qui est coûteuse et potentiellement toxique. Il est donc crucial d'identifier des biomarqueurs capables de prédire la réponse au traitement.

Les lymphocytes infiltrant la tumeur (TILs) sont reconnus comme un biomarqueur pronostique important, mais leur évaluation manuelle est subjective, chronophage et souffre d'une variabilité inter-observateurs considérable.

Ces limitations motivent le développement d'approches automatisées basées sur l'intelligence artificielle pour quantifier et caractériser les TILs de manière plus objective et reproductible.

# Défis Spécifiques au Mélanome

- Cellules de mélanome mimétiques imitant d'autres types cellulaires
- Modèles génériques (HoverNet sur PanNuke) : performance sous-optimale
- Confusion fréquente entre cellules tumorales/lymphocytes/stromales

→ Nécessité d'un modèle spécifique au mélanome

L'analyse histopathologique du mélanome présente des défis spécifiques qui compliquent l'application des modèles d'IA existants :

1. Les cellules de mélanome ont une capacité unique à imiter morphologiquement d'autres types cellulaires, rendant leur identification automatique particulièrement difficile.

2. Les modèles génériques comme HoverNet, même lorsqu'ils sont pré-entraînés sur des datasets comme PanNuke (qui inclut des échantillons de peau), montrent des performances sous-optimales sur les images de mélanome.

3. Ces modèles confondent fréquemment les cellules tumorales avec des lymphocytes ou des cellules stromales, compromettant ainsi l'évaluation précise des TILs.

4. Les méthodes manuelles actuelles souffrent également d'une grande variabilité dans l'évaluation de la localisation des TILs (intratumorale vs péritumorale), qui est pourtant cruciale pour évaluer leur valeur pronostique.

Ces défis justifient le développement d'un modèle spécifiquement adapté au mélanome, entraîné sur des données annotées de haute qualité.

# Le Jeu de Données PUMA

- 155 ROIs de mélanomes primaires + 155 ROIs métastatiques
- Images H&E à 40x (1024×1024 pixels)
- Annotations validées par un dermato-pathologiste
- Noyaux : tumeur, lymphocytes, plasmocytes, histiocytes...
- Tissus : tumeur, stroma, épiderme, vaisseaux sanguins...

Le dataset PUMA (Panoptic segmentation of nUclei and tissue in MelanomA) constitue la première base de données spécifique au mélanome pour la segmentation panoptique des noyaux cellulaires et des tissus :

- Il comprend 155 régions d'intérêt (ROIs) de mélanomes primaires et 155 de mélanomes métastatiques, offrant ainsi un échantillonnage représentatif de la diversité morphologique de cette pathologie.

- Les images sont des coupes histologiques colorées à l'hématoxyline et à l'éosine (H&E), numérisées à un grossissement de 40x avec une résolution de 1024×1024 pixels.

- Chaque ROI est accompagnée d'une ROI contextuelle de 5120×5120 pixels centrée autour d'elle, permettant d'analyser l'environnement tissulaire plus large.

- Les annotations des noyaux cellulaires identifient 10 types distincts : tumeur, lymphocytes, plasmocytes, histiocytes, mélanophages, neutrophiles, cellules stromales, épithélium, endothélium et cellules apoptotiques.

- Les annotations tissulaires comprennent 5 catégories : tumeur, stroma, épithélium, vaisseaux sanguins et zones nécrotiques.

- Toutes les annotations ont été réalisées par un professionnel médical et vérifiées par un dermato-pathologiste certifié, garantissant leur qualité et leur pertinence clinique.

Ce jeu de données unique permet de développer et d'évaluer des modèles spécifiques au mélanome avec un niveau de granularité sans précédent.

# Prétraitement des Données

1. Chargement des images TIFF et annotations GeoJSON
2. Conversion des polygones en masques binaires
3. Format multi-canaux : [RGB, tissue, nuclei]
4. Découpage en patches 270×270
5. Séparation train/test (80%/20%)

Notre pipeline de prétraitement transforme les données brutes PUMA en un format adapté à l'entraînement du modèle HoverNet :

1. **Chargement des données sources** : Les images histologiques sont au format TIFF (1024×1024 pixels, coloration H&E). Les annotations des noyaux cellulaires et des tissus sont fournies au format GeoJSON, sous forme de polygones géoréférencés avec des attributs de classification.

2. **Conversion des annotations** : Notre script `preprocess_data.py` transforme les polygones GeoJSON en masques binaires. Pour les noyaux, nous créons à la fois un masque d'instance (où chaque noyau a un identifiant unique) et un masque de type (où chaque pixel est codé selon le type cellulaire).

3. **Création de tableaux multi-canaux** : Nous combinons l'image RGB avec les masques de tissu et de noyaux pour créer un tableau numpy à 5 canaux : [R, G, B, tissue_type, nuclei_type].

4. **Découpage en patches** : Pour faciliter l'entraînement et augmenter le nombre d'échantillons, nous découpons chaque image en patches de 270×270 pixels.

5. **Division en ensembles** : Les patches sont répartis aléatoirement entre ensemble d'entraînement (80%) et de test (20%), en veillant à maintenir une distribution équilibrée des classes.

6. **Visualisation** : Pour vérifier la qualité du prétraitement, nous générons des visualisations superposant les masques de segmentation sur les images originales.

Cette phase de prétraitement est essentielle pour garantir que les données sont dans un format approprié et cohérent pour l'entraînement de notre modèle.

# Analyse du Script de Prétraitement

Notre script `preprocess_data.py` réalise les opérations suivantes :

## 1. Définition des mappings et constantes

```python
NUCLEI_MAP = {
    "background": 0,
    "nuclei_tumor": 1,
    "nuclei_stroma": 2,
    "nuclei_epithelium": 3,
    "nuclei_histiocyte": 4,
    "nuclei_melanophage": 5,
    "nuclei_neutrophil": 6,
    "nuclei_lymphocyte": 7,
    "nuclei_plasma_cell": 8,
    "nuclei_endothelium": 9,
    "nuclei_apoptosis": 10,
}

TISSUE_MAP = {
    "background": 0,
    "tissue_tumor": 1,
    "tissue_stroma": 2,
    "tissue_epithelium": 3,
    "tissue_blood_vessel": 4,
    "tissue_necrosis": 5,
    "tissue_epidermis": 6,
    "tissue_white_background": 7,
}
```

Nous définissons des dictionnaires qui associent chaque type cellulaire et tissulaire à une valeur numérique unique, utilisée pour coder les masques de segmentation.

## 2. Validation des données GeoJSON avec Pydantic

```python
class GeometryPolygon(BaseModel):
    type: Literal["Polygon"]
    coordinates: list[list[list[float]]]

class Classification(BaseModel):
    name: str
    color: list[int]

class GeoJSONData(BaseModel):
    type: str = Field("FeatureCollection")
    features: List[Feature]
```

Nous utilisons Pydantic pour valider la structure des fichiers GeoJSON et faciliter l'extraction des informations nécessaires, ce qui garantit la robustesse du prétraitement face à des données potentiellement mal formatées.

## 3. Conversion des polygones en masques

```python
def geojson_to_mask(geojson_path, image_shape, mapping):
    mask = np.zeros(image_shape[:2], dtype=np.int8)
    data = load_geojson(geojson_path)
    
    for feature in data.features:
        # Extraire les coordonnées du polygone
        if feature.geometry.type == "Polygon":
            polygons = [feature.geometry.coordinates[0]]
        elif feature.geometry.type == "MultiPolygon":
            polygons = [poly[0] for poly in feature.geometry.coordinates]
        
        for polygon_coords in polygons:
            points = np.array(polygon_coords, dtype=np.int32)
            type_int = mapping[feature.properties.classification.name]
            cv2.fillPoly(mask, [points], type_int)
    
    return mask
```

Cette fonction centrale convertit les coordonnées des polygones GeoJSON en masques binaires. Nous utilisons OpenCV (`cv2.fillPoly`) pour remplir efficacement chaque polygone avec la valeur correspondant à son type cellulaire ou tissulaire.

## 4. Extraction et sauvegarde des patches

```python
def extract_patches(image, patch_size):
    height, width = image.shape[:2]
    patches = []
    
    # Extract patches with a sliding window approach
    for y in range(0, height - patch_size[0] + 1, patch_size[0]):
        for x in range(0, width - patch_size[1] + 1, patch_size[1]):
            if len(image.shape) == 3:  # RGB image
                patch = image[y:y+patch_size[0], x:x+patch_size[1], :]
            else:  # Mask (2D)
                patch = image[y:y+patch_size[0], x:x+patch_size[1]]
            
            patches.append((patch, (y, x)))
    
    return patches
```

Nous découpons les images et les masques en patches de taille fixe (270×270 pixels) avec une approche de fenêtre glissante, en conservant les coordonnées pour maintenir la traçabilité spatiale.

## 5. Traitement en parallèle

```python
# Process images in parallel with ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=args.max_threads) as executor:
    futures = {
        executor.submit(
            process_image,
            image_path,
            args.nucleis,
            args.tissues,
            args.output,
        ): image_path
        for image_path in image_files
    }
```

Pour optimiser le temps de traitement, nous utilisons `ThreadPoolExecutor` qui permet de traiter plusieurs images simultanément, ce qui est crucial pour les grands jeux de données comme PUMA qui contient 310 ROIs.

## 6. Visualisation des masques générés

```python
def visualize_masks(img, tissue_mask, nuclei_mask, basename, output_dir):
    # Create visualization directory
    vis_dir = output_dir / "visualizations"
    vis_dir.mkdir(parents=True, exist_ok=True)
    
    # Create a figure with three panels
    figure, axes = plt.subplots(1, 3, figsize=(12, 4), dpi=100)
    
    axes[0].imshow(img)
    axes[0].set_title("Original image")
    
    plot_overlay(img, tissue_mask, axes[1], TISSUE_COLOR_MAP, TISSUE_MAP)
    axes[1].set_title("Tissue overlay")
    
    plot_overlay(img, nuclei_mask, axes[2], NUCLEI_COLOR_MAP, NUCLEI_MAP)
    axes[2].set_title("Nuclei overlay")
```

Cette fonction génère des visualisations qui permettent de vérifier la qualité du prétraitement en superposant les masques de segmentation (colorés selon le type cellulaire ou tissulaire) sur les images originales.

# Architecture HoverNet

- Réseau de neurones convolutif avec encodeur-décodeur
- Sorties multiples :
  - Segmentation des instances (noyaux individuels)
  - Classification des types cellulaires
  - Cartes de distances horizontales et verticales (HoVer-maps)

→ Permet simultanément de détecter, segmenter et classifier les noyaux

HoverNet est une architecture de deep learning spécialement conçue pour la segmentation et la classification simultanées des noyaux cellulaires dans les images histopathologiques :

- **Architecture de base** : HoverNet utilise un réseau de neurones à convolution avec une structure encodeur-décodeur, inspirée de U-Net. L'encodeur extrait des caractéristiques de plus en plus abstraites, tandis que le décodeur reconstitue progressivement les détails spatiaux.

- **Particularité des HoVer-maps** : L'innovation majeure de HoverNet réside dans l'utilisation de cartes de distances horizontales et verticales (HoVer pour Horizontal-Vertical). Ces cartes encodent pour chaque pixel sa distance relative aux bords des noyaux dans les directions horizontale et verticale, facilitant ainsi la séparation des noyaux qui se touchent.

- **Branches multiples** : L'architecture comporte trois branches parallèles en sortie du décodeur :
  1. Une branche de segmentation binaire (noyau vs fond)
  2. Une branche de cartes HoVer pour la séparation des instances
  3. Une branche de classification pour prédire le type de chaque noyau

- **Avantages** : Cette approche permet de résoudre simultanément trois problèmes difficiles : la détection des noyaux, leur segmentation individuelle (même lorsqu'ils sont agglutinés), et leur classification par type cellulaire.

- **HoverNeXt** : Version améliorée qui intègre des techniques d'augmentation de données plus sophistiquées et des mécanismes d'inférence adaptative pour mieux gérer les variations de coloration et de morphologie entre les échantillons.

Ces caractéristiques font de HoverNet une architecture particulièrement adaptée au challenge de la segmentation panoptique dans les images histopathologiques du mélanome.